In [1]:
from pyspark.sql import SparkSession
import pyspark
from pyspark import SparkContext

sc =SparkContext.getOrCreate()
from pyspark.sql import SQLContext
trurl='/home/aaftab/Dropbox/Techsoc/train_techsoc.csv'
tsturl='/home/aaftab/Dropbox/Techsoc/test_techsoc.csv'

In [2]:
sc.addFile(trurl)
sc.addFile(tsturl)

sqlcontext=SQLContext(sc)

In [3]:
from pyspark import SparkFiles

In [4]:
trdf=sqlcontext.read.csv(SparkFiles.get('train_techsoc.csv'), header=True, inferSchema=True)

In [5]:
tstdf=sqlcontext.read.csv(SparkFiles.get('test_techsoc.csv'), header=True, inferSchema=True)

In [6]:
trdf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- epoch: string (nullable = true)
 |-- sat_id: integer (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)
 |-- Vx: double (nullable = true)
 |-- Vy: double (nullable = true)
 |-- Vz: double (nullable = true)
 |-- x_sim: double (nullable = true)
 |-- y_sim: double (nullable = true)
 |-- z_sim: double (nullable = true)
 |-- Vx_sim: double (nullable = true)
 |-- Vy_sim: double (nullable = true)
 |-- Vz_sim: double (nullable = true)



In [7]:
trdf.show(5, truncate=False)

+---+-----------------------+------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|id |epoch                  |sat_id|x                  |y                 |z                  |Vx                 |Vy                |Vz                 |x_sim              |y_sim              |z_sim              |Vx_sim             |Vy_sim            |Vz_sim             |
+---+-----------------------+------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|0  |2014-01-01T00:00:00.000|0     |-20551.542729729616|30442.183965740267|-48103.896802169125|-2.1078805534520573|-8.838164531369227|-4.692610127789226 |-20522.087696771745|3048

In [29]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['x_sim','y_sim','z_sim','Vx_sim','Vy_sim','Vz_sim','sat_id'], outputCol = 'features')
trd=vectorAssembler.transform(trdf)
trd=trd.select(['features','Vy'])
trd.show(5)             

+--------------------+------------------+
|            features|                Vy|
+--------------------+------------------+
|[-20522.087696771...|-8.838164531369227|
|[-24495.949107598...|-9.915381027376242|
|[-24533.911749531...|-9.393007872226981|
|[-21233.351704453...|-7.829425025207068|
|[-15616.695660273...|-5.847493579330634|
+--------------------+------------------+
only showing top 5 rows



In [26]:
from pyspark.ml.regression import GBTRegressor

lr = GBTRegressor(featuresCol = 'features', labelCol='Vy')
lr_model = lr.fit(trd)


In [27]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['x_sim','y_sim','z_sim','Vx_sim','Vy_sim','Vz_sim'], outputCol = 'features')
tstd=vectorAssembler.transform(tstdf)
tstd=tstd.select(['features'])
tstd.show(5)

+--------------------+
|            features|
+--------------------+
|[-67817.397487337...|
|[-69478.899261576...|
|[-69610.589087625...|
|[-68136.264984711...|
|[-64959.510617139...|
+--------------------+
only showing top 5 rows



In [28]:
import pandas as pd

lr_pred=lr_model.transform(tstd)
lr_pred.toPandas().to_csv('Vypred.csv')